In [38]:
import numpy as np
import seaborn
import matplotlib.pyplot as plt

In [39]:
# read data from json file

import json

data = json.load(open('training_data/training_data_2024_04_26.json'))

len(data.keys())

32

In [40]:
data_old = json.load(open('training_data/training_data.json'))

keys_old = list(data_old.keys())

In [41]:
keys = list(data.keys())

keys_new = [key for key in keys if key not in keys_old]

len(keys_new)

11

In [42]:
# create training set and test set

train_set = []
posture_list = ['back','left','right']

posture_list_id = {'back':0,'left':1,'right':2}

for user in keys_old:
    for posture in posture_list:
        for i in range(3):
            train_set.append({'x':np.array(data[user][posture][i]), 'y':np.array(posture_list_id[posture])})

test_set = []

for user in keys_new:
    for posture in posture_list:
        for i in range(3):
            test_set.append({'x':np.array(data[user][posture][i]), 'y':np.array(posture_list_id[posture])})

In [43]:
train_set2 = [{'x':x['x'].reshape(-1), 'y':x['y']} for x in train_set]
test_set2 = [{'x':x['x'].reshape(-1), 'y':x['y']} for x in test_set]

In [44]:
len(test_set2)

99

In [45]:
# lightgbm without data augmentation

# old data training set, new data test set

import lightgbm as lgb

model = lgb.LGBMClassifier(objective='multiclass')
model.fit([x['x'] for x in train_set2], [x['y'] for x in train_set2])
y_pred = model.predict([x['x'] for x in test_set2], [x['y'] for x in test_set2])

print('Accuracy of lightgbm classifier on test set: {:.2f}'.format(model.score([x['x'] for x in test_set2], [x['y'] for x in test_set2])))

Accuracy of lightgbm classifier on test set: 0.78


In [46]:
# load the data

data = np.load('training_data/training_data_arg2.npz', allow_pickle=True)

train_set = data['train_set']
test_set = data['test_set']
val_set = data['val_set']

train_set = [{'x':x['x'].reshape(-1), 'y':x['y']} for x in train_set]
val_set = [{'x':x['x'].reshape(-1), 'y':x['y']} for x in val_set]
test_set = [{'x':x['x'].reshape(-1), 'y':x['y']} for x in test_set]

train_set2 = train_set + val_set

print(len(train_set2))

1983


In [47]:
# lightgbm

import lightgbm as lgb

model = lgb.LGBMClassifier(objective='multiclass')
model.fit([x['x'] for x in train_set2], [x['y'] for x in train_set2])
y_pred = model.predict([x['x'] for x in test_set2], [x['y'] for x in test_set2])

print('Accuracy of lightgbm classifier on test set: {:.2f}'.format(model.score([x['x'] for x in test_set2], [x['y'] for x in test_set2])))

Accuracy of lightgbm classifier on test set: 0.39


In [48]:
 # read data from json file

import json

data = json.load(open('training_data/training_data_2024_04_26.json'))

len(data.keys())

32

In [49]:
# select 6 random user for test set and rest for training set

np.random.seed(0)

test_users = np.random.choice(list(data.keys()), 9, replace=False)

# select 3 random user for validation set and rest for training set

val_users = np.random.choice([user for user in data.keys() if user not in test_users], 5, replace=False)

train_users = [user for user in data.keys() if user not in test_users]

train_users = [user for user in train_users if user not in val_users]

print('test_users: ', test_users)
print('val_users: ', val_users)
print('train_users: ', train_users)

test_users:  ['EtAUg' 'EJBUw' '0ZAVg' 'EtFVg' '0RCVw' 'ERHUw' '0NDUg' 'EVCVg' 'UJKXA']
val_users:  ['ENEVQ' 'ktEVA' 'ENCVw' 'URAUw' 'kdDVQ']
train_users:  ['EVDUQ', 'EtEUA', '0BLVw', 'UtGVw', 'EtKUA', 'kJBUQ', 'EJKVg', '0REVg', 'UFHVw', 'EBHUg', 'UdAUA', 'EBCXQ', 'EtCVg', '0NHVg', 'URGXA', 'URDXQ', 'UBHVg', 'UJEUQ']


In [50]:
# create training set and test set

train_set = []
posture_list = ['back','left','right']

posture_list_id = {'back':0,'left':1,'right':2}

for user in train_users:
    for posture in posture_list:
        for i in range(3):
            train_set.append({'x':np.array(data[user][posture][i]), 'y':np.array(posture_list_id[posture])})

test_set = []

for user in test_users:
    for posture in posture_list:
        for i in range(3):
            test_set.append({'x':np.array(data[user][posture][i]), 'y':np.array(posture_list_id[posture])})

val_set = []

for user in val_users:
    for posture in posture_list:
        for i in range(3):
            val_set.append({'x':np.array(data[user][posture][i]), 'y':np.array(posture_list_id[posture])})

In [51]:
import numpy as np

train_set_arg = []

range_shift = 7
plot = True
j = 1

for n in range(-range_shift,range_shift+1):
    for i in train_set:
        img = i['x']
        min_id = np.argmin(np.mean(img, axis=0))  # Encontre a coluna com a menor temperatura média
        # converta os valores menores que a mediana dessa coluna para o valor da mediana
        img[:, min_id] = np.where(img[:, min_id] < np.median(img[:, min_id]), np.median(img[:, min_id]), img[:, min_id])
        min_id = np.argmin(np.mean(img, axis=0))
        shifted = np.roll(img, n, axis=1)  # Mova a imagem horizontalmente
        if n > 0:
            shifted[:, :n] = img[:, min_id:min_id+1]    # 
            shifted[:, :n] = shifted[:, :n] + np.random.normal(0, 0.25, shifted[:, :n].shape)
        else:
            shifted[:, n:] = img[:, min_id:min_id+1]  
            shifted[:, n:] = shifted[:, n:] + np.random.normal(0,0.25, shifted[:, n:].shape)
        # verifica se as três primeiras colunas tem temperatura maior que a mediana se sim, não adiciona a imagem
        if np.all(shifted[:, :4] > np.median(shifted)):
            continue
        # # verifica se as três últimas colunas tem temperatura maior que a mediana se sim, não adiciona a imagem
        if np.all(shifted[:, -4:] > np.median(shifted)):
            continue
        train_set_arg.append({'x': shifted, 'y': i['y']})  # Adicione a imagem modificada ao conjunto de teste

train_set = train_set_arg.copy()

val_set_arg = []

for n in range(-range_shift,range_shift+1):
    for i in val_set:
        img = i['x']
        min_id = np.argmin(np.mean(img, axis=0))  # Encontre a coluna com a menor temperatura média
        # converta os valores menores que a mediana dessa coluna para o valor da mediana
        img[:, min_id] = np.where(img[:, min_id] < np.median(img[:, min_id]), np.median(img[:, min_id]), img[:, min_id])
        min_id = np.argmin(np.mean(img, axis=0))
        shifted = np.roll(img, n, axis=1)  # Mova a imagem horizontalmente
        if n > 0:
            shifted[:, :n] = img[:, min_id:min_id+1]    # 
            shifted[:, :n] = shifted[:, :n] + np.random.normal(0, 0.25, shifted[:, :n].shape)
        else:
            shifted[:, n:] = img[:, min_id:min_id+1]  
            shifted[:, n:] = shifted[:, n:] + np.random.normal(0,0.25, shifted[:, n:].shape)

        # verifica se as três primeiras colunas tem temperatura maior que a mediana se sim, não adiciona a imagem
        if np.all(shifted[:, :4] > np.median(shifted)):
            continue
        # verifica se as três últimas colunas tem temperatura maior que a mediana se sim, não adiciona a imagem
        if np.all(shifted[:, -4:] > np.median(shifted)):
            continue

        val_set_arg.append({'x': shifted, 'y': i['y']})  # Adicione a imagem modificada ao conjunto de teste

val_set = val_set_arg.copy()

In [52]:
np.savez('training_data/training_data_arg3.npz', train_set=train_set_arg, test_set=test_set, val_set=val_set_arg)